In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#Loading the dataSet

In [ ]:
data = pd.read_csv('Simple_Dataset.csv')

In [6]:
print(data.head())
print(data.info())

   CustomerID  Tenure  MonthlyCharges  TotalCharges        Contract  \
0        1001       5            70.0         350.0  Month-to-month   
1        1002      10            85.5         850.5        Two year   
2        1003       3            55.3         165.9        One year   
3        1004       8            90.0         720.0  Month-to-month   
4        1005       2            65.2         130.4        One year   

      PaymentMethod  Churn  
0  Electronic check      1  
1      Mailed check      0  
2  Electronic check      1  
3       Credit card      0  
4  Electronic check      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerID      5 non-null      int64  
 1   Tenure          5 non-null      int64  
 2   MonthlyCharges  5 non-null      float64
 3   TotalCharges    5 non-null      float64
 4   Contract        5 non-nu

In [7]:
data = data.drop(columns=['CustomerID']) #Simplify the dataset
data = data.dropna()  # Simple example of dropping missing values

In [9]:
print(data.head())

   Tenure  MonthlyCharges  TotalCharges        Contract     PaymentMethod  \
0       5            70.0         350.0  Month-to-month  Electronic check   
1      10            85.5         850.5        Two year      Mailed check   
2       3            55.3         165.9        One year  Electronic check   
3       8            90.0         720.0  Month-to-month       Credit card   
4       2            65.2         130.4        One year  Electronic check   

   Churn  
0      1  
1      0  
2      1  
3      0  
4      1  


In [10]:
data = pd.get_dummies(data, drop_first=True)

In [11]:
print(data.head())

   Tenure  MonthlyCharges  TotalCharges  Churn  Contract_One year  \
0       5            70.0         350.0      1              False   
1      10            85.5         850.5      0              False   
2       3            55.3         165.9      1               True   
3       8            90.0         720.0      0              False   
4       2            65.2         130.4      1               True   

   Contract_Two year  PaymentMethod_Electronic check  \
0              False                            True   
1               True                           False   
2              False                            True   
3              False                           False   
4              False                            True   

   PaymentMethod_Mailed check  
0                       False  
1                        True  
2                       False  
3                       False  
4                       False  


In [12]:
X = data.drop('Churn', axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\dhira\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 18.4360 - val_accuracy: 0.0000e+00 - val_loss: 46.1935
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step - accuracy: 0.5000 - loss: 17.7749 - val_accuracy: 0.0000e+00 - val_loss: 49.6534
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7500 - loss: 3.6095 - val_accuracy: 0.0000e+00 - val_loss: 55.4443
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.5000 - loss: 9.1591 - val_accuracy: 0.0000e+00 - val_loss: 62.6399
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.7500 - loss: 27.0884 - val_accuracy: 0.0000e+00 - val_loss: 63.2564
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.5000 - loss: 13.8304 - val_accuracy: 0.0000e+00 - val_loss: 60.3565
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.7500 - loss: 28.7921 - val_accuracy: 0.0000e+00 - val_loss: 54.4476
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 1.0000 - loss: 0.0345

In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

1/1 - 0s - 68ms/step - accuracy: 0.0000e+00 - loss: 34.8610
Test accuracy: 0.0


In [25]:
saved_model_dir = "saved_model_churn"
model.export(saved_model_dir)   # <- Use export(), not save()

# 2. (Optional) Save a backup in HDF5
model.save("churn_model.h5")

# 3. Convert SavedModel → TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# 4. Save TFLite file
with open("model_churn.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model written to model_churn.tflite")

INFO:tensorflow:Assets written to: saved_model_churn\assets


INFO:tensorflow:Assets written to: saved_model_churn\assets


Saved artifact at 'saved_model_churn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2312385144080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2312385143888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2312385145232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2312385146000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2312385145616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2312385145040: TensorSpec(shape=(), dtype=tf.resource, name=None)


TFLite model written to model_churn.tflite
